<a href="https://colab.research.google.com/github/werowe/HypatiaAcademy/blob/master/stats/consolidate_weather_data_run_sql_spark_queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd

# Define the directory containing the CSV files
directory = 'drive/MyDrive/weather'

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with 'paphos2024' and ends with '.csv'
    if filename.startswith('paphos20') and filename.endswith('.csv'):
        # Read the CSV file into a DataFrame
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        dataframes.append(df)

# Combine all DataFrames into one
df_combined = pd.concat(dataframes, ignore_index=True)

# Drop duplicate rows
df_combined = df_combined.drop_duplicates()

# Save the combined DataFrame to a new CSV file (optional)
df_combined.to_csv('combined_weather.csv', index=False)

# Print a summary of the combined DataFrame
print(f"Combined DataFrame shape: {df_combined.shape}")

Combined DataFrame shape: (11200, 24)


In [3]:
from pyspark.sql import SparkSession


# Initialize SparkSession
spark = SparkSession.builder \
    .appName("weather") \
    .getOrCreate()

df = spark.read.csv(
    "combined_weather.csv",
    header=True,        # Use the first row as column names
    inferSchema=True,   # Automatically infer data types
    sep=",",            # Specify delimiter (default is ',')
    encoding="UTF-8"    # Handle encoding
)



In [4]:
df.columns

['name',
 'datetime',
 'temp',
 'feelslike',
 'dew',
 'humidity',
 'precip',
 'precipprob',
 'preciptype',
 'snow',
 'snowdepth',
 'windgust',
 'windspeed',
 'winddir',
 'sealevelpressure',
 'cloudcover',
 'visibility',
 'solarradiation',
 'solarenergy',
 'uvindex',
 'severerisk',
 'conditions',
 'icon',
 'stations']

In [5]:
df.createOrReplaceTempView("weather")

In [6]:

sql = '''
SELECT SUM(precip) AS total_precip, YEAR(datetime) AS year, MONTH(datetime) AS month
FROM weather
GROUP BY YEAR(datetime), MONTH(datetime)

'''

result = spark.sql(sql)
result.show()

+------------------+----+-----+
|      total_precip|year|month|
+------------------+----+-----+
|               0.0|2024|    7|
|486.78400000000016|2024|   12|
|1.6430000000000005|2024|    9|
|1.5000000000000002|2024|   10|
|            23.965|2024|    1|
|166.32799999999978|2024|   11|
| 7.687999999999999|2025|    1|
|             0.008|2024|    8|
+------------------+----+-----+



In [7]:
sql = '''
SELECT round(avg(temp),2) AS temp, YEAR(datetime) AS year, MONTH(datetime) AS month
FROM weather
GROUP BY YEAR(datetime), MONTH(datetime)
sort by month

'''

result = spark.sql(sql)
result.show()

+-----+----+-----+
| temp|year|month|
+-----+----+-----+
|15.28|2024|    1|
|14.41|2025|    1|
|29.12|2024|    7|
|28.42|2024|    8|
|26.56|2024|    9|
|22.68|2024|   10|
|18.29|2024|   11|
|15.07|2024|   12|
+-----+----+-----+



In [8]:
sql = '''
SELECT avg(temp) AS temp, YEAR(datetime) AS year
FROM weather
GROUP BY YEAR(datetime)


'''

result = spark.sql(sql)
result.show()

+------------------+----+
|              temp|year|
+------------------+----+
|14.406673209028435|2025|
|22.018947058245622|2024|
+------------------+----+



In [26]:
sql = '''
SELECT YEAR(datetime) AS year, MONTH(datetime) AS month, AVG(temp) AS avg_temp
FROM weather
WHERE YEAR(datetime) IN (2025, 2024) AND MONTH(datetime) = 1
GROUP BY YEAR(datetime), MONTH(datetime);

'''

result = spark.sql(sql)
result.show()

+----+-----+------------------+
|year|month|          avg_temp|
+----+-----+------------------+
|2024|    1|15.282291666666676|
|2025|    1|14.406673209028435|
+----+-----+------------------+

